In [18]:
!pip install PyPDF2
!pip install python-docx
!pip install find-job-titles
!pip install Levenshtein
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
!sudo apt-get update -y
!sudo apt-get install -y antiword

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
antiword is already the newest version (0.37-16).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.


#  This section converts every document type into strings (txt)


(Docx , HTML, PDF , Doc)

In [20]:
from bs4 import BeautifulSoup
from docx import Document
from PyPDF2 import PdfReader

def identify_file_type(file_path):
    resume = ""

    file_name = file_path.split('/')[-1]  # Extracting the file name from the path
    if file_name.endswith('.pdf'):
        reader = PdfReader(file_path)
        for i in range(len(reader.pages)):
            page = reader.pages[i]
            text = page.extract_text()
            resume += text
        return resume

    elif file_name.endswith('.docx'):
        document = Document(file_path)
        for p in document.paragraphs:
            resume += p.text
        return resume

    elif file_name.endswith('.html'):
        with open(file_path, 'r', encoding='utf-8') as html_file:
            soup = BeautifulSoup(html_file, 'html.parser')
            for line in soup.get_text().splitlines():
                resume += line
        return resume

    elif file_name.endswith('.doc'):
      try:
        process = subprocess.Popen(['antiword', file_path], stdout=subprocess.PIPE)
        output, _ = process.communicate()
        return output.decode('utf-8')

      except Exception as e:
        print("Error:", e)
    else:
        return 'Unknown File Type'

# Example usage:
file_path = "/content/documents20220826-1-v01cla.pdf"  # Example file path
resume = identify_file_type(file_path)
print("File type:", resume)

File type: Ashwini Kalidas Vedula
Email ID: ashwini.vedula@nyu.edu |+1 (201)-616-1347 |LinkedIn: linkedin.com/in/ashwini-vedula
Education
New York University, New York, NY Sept 2021- May 2023
Courant Institute of Mathematical Sciences and Stern School of Business
Master of Science in Information Systems - 3.6/4
Courses: Dealing with Data, Data Science for Business : Technical, Database Management Systems, Real Time
Big Data Analytics, Cloud Computing
Technical Skills
Databases & Tools : MySQL, SSMS, MS Access, PostgreSQL, PL/SQL, MongoDB
Data Analysis & Visualization Tools : Python, R , SQL, MS Excel, Tableau, Power BI
Cloud Platforms & Big Data Technologies : AWS (Redshift, EC2, S3, EMR, SageMaker, DynamoDB Table), Azure
(Storage, IoT, Virtual Machines), GCP (BigTable, DataLab), Apache Hadoop, Hive
Experience
Clark Associates Inc Pennsylvania, USA
Financial Systems and Data Intern May 2022 - Aug 2022
•Extracted data from varied data sources to automate Summary Income financial reports

# This section imports the pretrained model we have used from hugging face

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("has-abi/distilBERT-finetuned-resumes-sections")
model = AutoModelForSequenceClassification.from_pretrained("has-abi/distilBERT-finetuned-resumes-sections")

# This section splits the text into paragraphs\lines

In [22]:
# Tokenize input
input_text = resume.split("\n")
print(input_text)

['Ashwini Kalidas Vedula', 'Email ID: ashwini.vedula@nyu.edu |+1 (201)-616-1347 |LinkedIn: linkedin.com/in/ashwini-vedula', 'Education', 'New York University, New York, NY Sept 2021- May 2023', 'Courant Institute of Mathematical Sciences and Stern School of Business', 'Master of Science in Information Systems - 3.6/4', 'Courses: Dealing with Data, Data Science for Business : Technical, Database Management Systems, Real Time', 'Big Data Analytics, Cloud Computing', 'Technical Skills', 'Databases & Tools : MySQL, SSMS, MS Access, PostgreSQL, PL/SQL, MongoDB', 'Data Analysis & Visualization Tools : Python, R , SQL, MS Excel, Tableau, Power BI', 'Cloud Platforms & Big Data Technologies : AWS (Redshift, EC2, S3, EMR, SageMaker, DynamoDB Table), Azure', '(Storage, IoT, Virtual Machines), GCP (BigTable, DataLab), Apache Hadoop, Hive', 'Experience', 'Clark Associates Inc Pennsylvania, USA', 'Financial Systems and Data Intern May 2022 - Aug 2022', '•Extracted data from varied data sources to au

# This section classifies the split strings into 12 predefined classes and prints the classified output

In [23]:
import re

model_classes = {
    0: "awards",
    1: "certificates",
    2: "contact/name/title",
    3: "education",
    4: "interests",
    5: "languages",
    6: "para",
    7: "professional_experiences",
    8: "projects",
    9: "skills",
    10: "soft_skills",
    11: "summary"
}

final_output = {}
skills_text = ""  # Initialize empty string to store skills text
skills_list = []  # Initialize empty list to store skills paragraphs


prof_pattern = r'\b(analyst|engineer|developer|intern|system|data)\b'

certifications = []
certification_patterns = [                              #Define a list of common certification abbreviations and patterns
        r"\b([A-Za-z0-9\s]+)\b certification\b",
        r"\bcertified (\S+)\b",
        r"\b([A-Za-z0-9\s]+)\b certified\b",
        r"\b([A-Za-z0-9\s]+)\b certificate\b"
    ]

for inp in input_text:
  inputs = tokenizer(inp, return_tensors = "pt")
  output = model(**inputs)

  output_logits = output.logits.tolist()
  max_op = max(max(output_logits))

  predicted_class_index = output_logits[0].index(max_op)
  predicted_class_label = model_classes[predicted_class_index]

  if predicted_class_label == "awards":
    final_output.setdefault("awards", []).append(inp)

  elif predicted_class_label == "certificates":
    if predicted_class_label == "certificates":
      extracted_certificates = []
      for pattern in certification_patterns:
        matches = re.findall(pattern, inp, re.IGNORECASE)
        if matches:
          extracted_certificates.extend(matches)
          final_output.setdefault("certificates", []).extend(extracted_certificates)

  elif predicted_class_label == "contact/name/title":
    if re.findall(prof_pattern, inp, re.IGNORECASE):
      final_output.setdefault("professional_experiences", []).append(inp)
    else:
      final_output.setdefault("contact/name/title", []).append(inp)

  elif predicted_class_label == "education":
        final_output.setdefault("education", []).append(inp)

  elif predicted_class_label == "interests":
        final_output.setdefault("interests", []).append(inp)

  elif predicted_class_label == "languages":
        final_output.setdefault("languages", []).append(inp)

  elif predicted_class_label == "para":
        final_output.setdefault("para", []).append(inp)

  elif predicted_class_label == "professional_experiences":
        final_output.setdefault("professional_experiences", []).append(inp)

  elif predicted_class_label == "projects":
        final_output.setdefault("projects", []).append(inp)

  elif predicted_class_label == "skills":
        final_output.setdefault("skills", []).append(inp)
        # Append paragraph to the skills list
        skills_list.append(inp)

  elif predicted_class_label == "soft_skills":
        final_output.setdefault("soft_skills", []).append(inp)

  elif predicted_class_label == "summary":
        final_output.setdefault("summary", []).append(inp)

# Join the skills list into a single string separated by commas
skills_text = ", ".join(skills_list)

for key, value in final_output.items():
    print(f"{key}: {value}")

contact/name/title: ['Ashwini Kalidas Vedula', 'Email ID: ashwini.vedula@nyu.edu |+1 (201)-616-1347 |LinkedIn: linkedin.com/in/ashwini-vedula', 'Technical Skills', 'Clark Associates Inc Pennsylvania, USA', 'Dacapo Brokerage India Private Limited Mumbai,India', 'CRISIL Limited, An S&P Global Company Mumbai,India', 'Management of CRISIL’s Business Development Operations Team.', 'Onsite Supply Chain Analytics Project - Doha, Qatar', 'conversions by 55%.', 'algorithm.']
education: ['Education', 'New York University, New York, NY Sept 2021- May 2023', 'Courant Institute of Mathematical Sciences and Stern School of Business', 'Master of Science in Information Systems - 3.6/4']
skills: ['Big Data Analytics, Cloud Computing', 'Databases & Tools : MySQL, SSMS, MS Access, PostgreSQL, PL/SQL, MongoDB', 'Data Analysis & Visualization Tools : Python, R , SQL, MS Excel, Tableau, Power BI', 'Cloud Platforms & Big Data Technologies : AWS (Redshift, EC2, S3, EMR, SageMaker, DynamoDB Table), Azure', '(S

In [24]:
import re

def remove_long_segments(text):
    segments = text.split(',')
    result = []
    for segment in segments:
        # Check if segment contains more than 4 words
        if len(segment.split()) > 3:
            # Check if the segment contains a colon, opening or closing parentheses, or an ampersand
            if re.search(r':[^(]*\)|\(&\)|&', segment):
                result.append(segment)
        else:
            result.append(segment)
    return ', '.join(result)

# Example usage
# ztext = "Big Data Analytics, Cloud Computing Databases & Tools : MySQL, SSMS, MS Access, PostgreSQL, PL/SQL, MongoDB Data Analysis & Visualization Tools : Python, R , SQL, MS Excel, Tableau, Power BI Cloud Platforms & Big Data Technologies : AWS (Redshift, EC2, S3, EMR, SageMaker, DynamoDB Table), Azure (Storage, IoT, Virtual Machines), GCP (BigTable, DataLab), Apache Hadoop, Hive metrics and performance measures."
filtered_text = remove_long_segments(skills_text)

skills = filtered_text.split(",")
for s in skills:
  print(s)

Big Data Analytics
  Cloud Computing
  Databases & Tools : MySQL
  SSMS
  MS Access
  PostgreSQL
  PL/SQL
  MongoDB
  Data Analysis & Visualization Tools : Python
  R 
  SQL
  MS Excel
  Tableau
  Power BI
  Cloud Platforms & Big Data Technologies : AWS (Redshift
  EC2
  S3
  EMR
  SageMaker
  DynamoDB Table)
  Azure
  (Storage
  IoT
  Virtual Machines)
  GCP (BigTable
  DataLab)
  Apache Hadoop
  Hive
  archiving
  stored


# Extract jobs from the resume

In [25]:
# from find_job_titles import Finder
# # Finding job titles
# finder = Finder()
# job_titles = finder.findall(resume)
# print("Job titles found:", job_titles)

from find_job_titles import Finder
import re
finder = Finder()
def find_jobs(resume):
  return(list(finder.findall(resume)))

def extract_job_strings(jobs):
  f_jobs = []
  job_pattern = r"'([^']*)'"
  for job in jobs:
    j = str(job)
    job_extracted = str(re.findall(job_pattern, j))
    if (job_extracted not in f_jobs):
      f_jobs.append(job_extracted)
    else:
      continue
  return f_jobs

def remove_extra_characters(input_string):
  cleaned_string = re.sub(r'\W+', ' ', input_string)
  return cleaned_string.strip()

jobs_from_resume = find_jobs(resume)
f_jobs = extract_job_strings(jobs_from_resume)
final_jobs = []
for j in f_jobs:
  final_jobs.append(remove_extra_characters(j))

print(final_jobs)

['Associate', 'Senior Analyst', 'Senior Associate', 'Business Development', 'Associate Software Engineer', 'Student']


# returning ONET job codes

In [26]:
import pandas as pd
df = pd.read_csv('/content/ONET_codes.csv')

for jobs in final_jobs:
    matching_rows = df[df['Resume Job Title'] == jobs]
    for row in matching_rows.itertuples(index=False):
        print(" - ".join(map(str, row[1:])))


Associate - Chief Executives - 11-1011.00
Senior Analyst - Sales Engineers - 41-9031.00
Senior Associate - Editors - 27-3041.00
Business Development - Financial Managers - 11-3031.00
Associate Software Engineer - Software Developers - 15-1252.00
Student - Tutors - 25-3041.00


# Creating a career trajectory

In [27]:
import re

def extract_dates(text):
    # Regular expression patterns to match dates in different formats
    date_pattern1 = r'(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s\d{4}'
    date_pattern2 = r'\b(?:19|20)\d{2}\b'

    # Check for matches using both patterns
    matches = re.findall(date_pattern1, text) + re.findall(date_pattern2, text)
    return matches

def extract_career_trajectory(resume_text):
    # Extract dates from the resume text
    experiences = resume_text.split("\n")

    # Extract the dates associated with each experience
    career_trajectory = []
    for experience in experiences:
        dates_in_experience = extract_dates(experience)
        if dates_in_experience:
            # Choose the first date encountered in each experience section
            start_date = min(dates_in_experience)
            career_trajectory.append((start_date, experience.strip()))

    # Sort the career trajectory by starting dates
    career_trajectory.sort(key=lambda x: x[0])

    # Extract only the career trajectory text
    career_trajectory_text = [experience[1] for experience in career_trajectory]

    return career_trajectory_text

# Example usage
# Extract career trajectory
career_trajectory = extract_career_trajectory(resume)

# Print career trajectory
for experience in career_trajectory:
    print(experience)


•Treasurer of Computer Society of India -VESIT Student Chapter for the year 2016-2017.
Associate Software Engineer - Data Science & Quants Lab Jul 2018 - Oct 2019
Senior Associate - Data Science & Quants Lab Oct 2019 - Jul 2020
•Conferred Quarterly Award ,(Service Excellence Award - Q4 2019) at Crisil Limited.
•Received CRISILite Award for Performance (CLAP) , for the month of May 2019.
Senior Analyst Aug 2020 - Jul 2021
New York University, New York, NY Sept 2021- May 2023
Financial Systems and Data Intern May 2022 - Aug 2022
